In [1]:
using CUDA
using TypedTables
using DataFrames
using Statistics
using CSV

In [14]:
# CUDA

function operation_gpu(A, B)
    C = CUDA.zeros(Float32, size(A))
    CUDA.@sync C = A * B
    return C
end

function simulate_gpu(A_normal, B_normal)
    # Convert your Float32 matrices to Float32 and then to CuArray{Float32}
    A = CUDA.convert(CuArray{Float32}, A_normal)
    B = CUDA.convert(CuArray{Float32}, B_normal)
    # Mengukur waktu eksekusi
    cuda_start = CUDA.@elapsed result_gpu = operation_gpu(A, B)
    # println("Waktu eksekusi dengan CUDA: $cuda_start detik")
    return cuda_start
end

simulate_gpu (generic function with 1 method)

In [15]:
# CPU

function operation_cpu(A, B)
    C = zeros(Float32, size(A))
    C = A * B
    return C
end

function simulate_cpu(A_normal, B_normal)
    normal_start = @elapsed result_cpu = operation_cpu(A_normal, B_normal)
    # println("Waktu eksekusi dengan CPU: $normal_start detik")
    return normal_start
end

simulate_cpu (generic function with 1 method)

In [16]:
function simulate(N::Int64, n_loop::Int64)
    println("Total size: $(N*N)")
    results_gpu = []
    results_cpu = []
    is_better_gpu = []
    for i in 1:n_loop
        A_normal = rand(Float32, N, N)
        B_normal = rand(Float32, N, N)
        benchmark_gpu = simulate_gpu(A_normal, B_normal)
        benchmark_cpu = simulate_cpu(A_normal, B_normal)
        push!(results_gpu, benchmark_gpu)
        push!(results_cpu, benchmark_cpu)
        push!(is_better_gpu, benchmark_gpu < benchmark_cpu)
    end
    println("Mean CPU: $(mean(results_cpu))")
    println("Mean GPU: $(mean(results_gpu))")
    return DataFrame(CPU=results_cpu, GPU=results_gpu, Better_GPU=is_better_gpu)
end

simulate (generic function with 1 method)

In [20]:
df = simulate(10, 10)
CSV.write("./1.csv", df)
df

Total size: 100
Mean CPU: 2.1719000000000003e-6
Mean GPU: 6.03936e-5


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,6.548e-6,0.000151552,false
2,2.424e-6,6.5536e-5,false
3,1.697e-6,5.1104e-5,false
4,1.825e-6,4.8128e-5,false
5,1.525e-6,4.7104e-5,false
6,1.52e-6,4.7104e-5,false
7,1.461e-6,4.6976e-5,false
8,1.47e-6,4.7104e-5,false
9,1.603e-6,4.8128e-5,false


In [24]:
df = simulate(50, 10)
CSV.write("./2.csv", df)
df

Total size: 2500
Mean CPU: 2.88459e-5
Mean GPU: 9.71872e-5


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,3.1027e-5,0.000177152,false
2,2.3951e-5,0.000116736,false
3,8.6114e-5,8.6016e-5,true
4,1.804e-5,8.8064e-5,false
5,2.2871e-5,7.6896e-5,false
6,2.4908e-5,8.192e-5,false
7,1.7488e-5,7.9872e-5,false
8,2.1475e-5,0.000104448,false
9,2.0318e-5,8.192e-5,false


In [25]:
df = simulate(100, 10)
CSV.write("./3.csv", df)
df

Total size: 10000
Mean CPU: 0.0004589816
Mean GPU: 9.81152e-5


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.00225168,0.000140352,true
2,0.000431572,0.000185344,true
3,0.000255873,0.000111712,true
4,0.000256908,8.3968e-5,true
5,0.000234081,7.8848e-5,true
6,0.00024398,7.68e-5,true
7,0.000231346,7.2704e-5,true
8,0.000234882,7.7824e-5,true
9,0.000229383,7.7824e-5,true


In [26]:
df = simulate(500, 10)
CSV.write("./4.csv", df)
df

Total size: 250000
Mean CPU: 0.0032832404000000003
Mean GPU: 0.0004160896


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.0033096,0.000554848,true
2,0.0021114,0.000408864,true
3,0.00209852,0.000382816,true
4,0.00258531,0.000388608,true
5,0.00274567,0.000389888,true
6,0.00220663,0.000390688,true
7,0.00363327,0.000400768,true
8,0.00260534,0.000470752,true
9,0.00256405,0.00038896,true


In [27]:
df = simulate(1000, 10)
CSV.write("./5.csv", df)
df

Total size: 1000000
Mean CPU: 0.017756165900000002
Mean GPU: 0.0034048571


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,0.0154194,0.00254166,true
2,0.017074,0.00261578,true
3,0.0319458,0.010678,true
4,0.0192265,0.00269469,true
5,0.0143915,0.00248198,true
6,0.0206225,0.00292861,true
7,0.015005,0.00248112,true
8,0.0157706,0.00252554,true
9,0.0140647,0.00259811,true


In [28]:
df = simulate(5000, 10)
CSV.write("./6.csv", df)
df

Total size: 25000000
Mean CPU: 1.2315838221
Mean GPU: 0.29779717


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,1.39156,0.2984,true
2,1.44284,0.29229,true
3,1.20652,0.293076,true
4,1.02825,0.299387,true
5,1.45917,0.292907,true
6,1.03024,0.29525,true
7,1.08469,0.294597,true
8,1.27591,0.310019,true
9,1.08626,0.296031,true


In [29]:
df = simulate(10000, 10)
CSV.write("./7.csv", df)
df

Total size: 100000000
Mean CPU: 25.7547176659
Mean GPU: 2.2115183


Row,CPU,GPU,Better_GPU
,Any,Any,Any
1,9.44689,2.0698,true
2,12.3255,2.06541,true
3,22.9768,2.10634,true
4,28.2775,2.10754,true
5,26.5898,2.17774,true
6,34.5192,2.08004,true
7,30.0181,2.69238,true
8,33.904,2.05572,true
9,32.8328,2.36222,true
